# 🧠 Alzheimer MRI Sınıflandırması: Transfer Learning (ResNet50)

Bu notebook, basit CNN modelinde karşılaşılan sınırlılıkları aşmak ve Alzheimer MRI görüntülerinde **ResNet50 tabanlı transfer öğrenme** ile yüksek doğruluk elde etmeyi amaçlamaktadır.

## 1. Kütüphaneler ve Modüller

In [ ]:

import os, sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.dataset import build_dataloaders
from src.transforms import base_train_transform, default_transform
from src.model import get_model
from src.losses import FocalLoss
from src.evaluate import evaluate_model, save_confusion_matrix, save_classification_report
from src.visualization import plot_training, generate_gradcam, plot_gradcam_on_image


## 2. Veri Yükleme ve Transformlar

In [ ]:

data_root = "../data/processed"
batch_size = 32

train_loader, val_loader, test_loader, meta = build_dataloaders(
    data_root=data_root,
    batch_size=batch_size,
    train_transform=base_train_transform,
    val_transform=default_transform,
    use_sampler=True
)

classes = meta["classes"]
class_weights = meta["class_weights"]
print("Classes:", classes)
print("Class distribution:", meta["class_distribution"])


## 3. Model Tanımlama (ResNet50)

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

model = get_model(
    model_name="resnet",
    num_classes=len(classes),
    pretrained=True,
    freeze_backbone=False
).to(device)

gradcam_layer = model.backbone.layer4[-1]


## 4. Loss, Optimizer ve Scheduler

In [ ]:

criterion = FocalLoss(alpha=class_weights.to(device), gamma=2.0)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5, verbose=True)


## 5. Eğitim Döngüsü

In [ ]:

history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
best_val_loss = float("inf")

for epoch in range(30):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total

    val_acc, val_loss, _, _ = evaluate_model(model, val_loader, criterion, device)
    scheduler.step(val_loss)

    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc)
    history["val_acc"].append(val_acc)

    print(f"Epoch [{epoch+1}/30] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


## 6. Sonuçların Kaydedilmesi

In [ ]:

os.makedirs("../outputs/models", exist_ok=True)
torch.save(model.state_dict(), "../outputs/models/alzheimer_resnet.pt")

plot_training(history, save_path="../outputs/figures/training_curves_resnet.png")

val_acc, val_loss, y_true, y_pred = evaluate_model(model, val_loader, criterion, device)
save_confusion_matrix(y_true, y_pred, classes, "../outputs/figures/confusion_matrix_resnet.png")
save_classification_report(y_true, y_pred, classes, "../outputs/reports/classification_report_resnet.txt")


## 7. Grad-CAM Yorumlanabilirlik

In [ ]:

sample_img, _ = next(iter(val_loader))
sample_img = sample_img[0].unsqueeze(0).to(device)

heatmap, pred_class = generate_gradcam(model, sample_img, conv_layer=gradcam_layer, device=device)

inv_transform = transforms.Compose([
    transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
                         std=[1/0.229, 1/0.224, 1/0.225]),
    transforms.ToPILImage()
])
original_img = inv_transform(sample_img.squeeze().cpu())

plot_gradcam_on_image(original_img, heatmap, "../outputs/figures/gradcam_resnet.png")


## 8. Sonuç ve Çıkarımlar

- ResNet50 ile transfer öğrenme, **WeightedRandomSampler + Focal Loss** sayesinde %97 doğruluğa ulaştı.  
- `VeryMildDemented` ve `ModerateDemented` sınıflarında belirgin gelişmeler görüldü.  
- Grad-CAM çıktıları, modelin klinik olarak anlamlı bölgelere odaklandığını gösterdi.

* **Genel Doğruluk:** %97.47
* **Sınıf Bazlı F1-Skorları:**
    * `MildDemented`: %98
    * `ModerateDemented`: %100 🏆
    * `NonDemented`: %97
    * `VeryMildDemented`: %97

    ### **Çıkarım:**
* `Training` ve `Validation` eğrileri arasındaki makas kapanmıştır.
* `Validation Accuracy` %97'lere ulaşarak modelin genelleme yeteneğinin zirveye ulaştığı kanıtlanmıştır.
* **`VeryMildDemented`** sınıfı için elde edilen **%97'lik F1-score**, en zorlu problemin başarıyla çözüldüğünü gösterir.
* `Confusion Matrix` ve `Classification Report`'a göre, model tüm sınıflar için dengeli ve güvenilir bir performans sergilemektedir.